## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

In [1]:
import tensorflow as tf

In [3]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
tf.enable_eager_execution()

### Collect Data

In [4]:
#from google.colab import drive
#drive.mount('/gdrive')

In [3]:
import pandas as pd
import numpy as np

In [6]:
p_data = pd.read_csv('Prices.csv')

### Check all columns in the dataset

In [7]:
p_data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [8]:
p_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [9]:
p_data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [10]:
p_data = p_data.drop(labels='date', axis=1)
p_data = p_data.drop(labels='symbol', axis=1)

In [11]:
p_data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [12]:
p_data.shape

(851264, 5)

### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [13]:
p_d_1000 = p_data.head(1000)

In [14]:
p_d_1000.shape

(1000, 5)

In [15]:
p_d_1000['volume'] = p_d_1000.volume / 1000000

In [16]:
p_d_1000.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


In [17]:
type(p_d_1000)

pandas.core.frame.DataFrame

### Divide the data into train and test sets

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X = p_d_1000.drop(labels='volume', axis=1)
Y = p_d_1000['volume']
X.shape
Y.shape

(1000, 4)

(1000,)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
print("X_train ", X_train.shape)
print("X_test ", X_test.shape)
print("Y_train ", y_train.shape)
print("Y_test ", y_test.shape)

X_train  (700, 4)
X_test  (300, 4)
Y_train  (700,)
Y_test  (300,)


In [21]:
type(X)
type(Y)

pandas.core.frame.DataFrame

pandas.core.series.Series

#### Convert Training and Test Data to numpy float32 arrays


In [53]:
train_x = np.float32(X_train)
print(train_x.dtype)
train_y = np.float32(y_train)
print(train_y.dtype)
test_x = np.float32(X_test)
print(test_x.dtype)
test_y = np.float32(y_test)
print(test_y.dtype)

float32
float32
float32
float32


### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [56]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_x_fit = sc.fit(train_x)
train_x = train_x_fit.transform(train_x)
test_x_fit = sc.fit(test_x)
test_x_std = test_x_fit.transform(test_x)

test_x_std.shape
test_y.shape

(300, 4)

(300,)

In [58]:
train_x = np.float32(train_x)
print(train_x.dtype)
train_y = np.float32(train_y)
print(train_y.dtype)
test_x = np.float32(test_x)
print(test_x.dtype)
test_y = np.float32(test_y)
print(test_y.dtype)

float32
float32
float32
float32


## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [60]:
#Declare Weights and Bias

w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [61]:
# y = wx + b

def prediction(x, w, b):
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [62]:
#Define Loss

def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [63]:
#Define Gradient Descent function

def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record math ops on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss w.r.t Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b


## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [64]:
train_x.dtype
type(train_y)
type(w)
type(b)

dtype('float32')

numpy.ndarray

tensorflow.python.framework.ops.EagerTensor

tensorflow.python.framework.ops.EagerTensor

In [65]:
#Training the model for 100 iterations

for i in range(100):
    
    w, b = train(train_x, train_y, w, b)
    if i % 5 == 0:
        print('Current Loss on iteration', i, loss(train_y, prediction(train_x, w, b)).numpy())

Current Loss on iteration 0 224.67271
Current Loss on iteration 5 219.31863
Current Loss on iteration 10 214.94376
Current Loss on iteration 15 211.36942
Current Loss on iteration 20 208.44868
Current Loss on iteration 25 206.06247
Current Loss on iteration 30 204.1128
Current Loss on iteration 35 202.51955
Current Loss on iteration 40 201.2179
Current Loss on iteration 45 200.15422
Current Loss on iteration 50 199.28517
Current Loss on iteration 55 198.57532
Current Loss on iteration 60 197.9949
Current Loss on iteration 65 197.52095
Current Loss on iteration 70 197.13365
Current Loss on iteration 75 196.81714
Current Loss on iteration 80 196.5585
Current Loss on iteration 85 196.34721
Current Loss on iteration 90 196.17462
Current Loss on iteration 95 196.03357


### Get the shapes and values of W and b

In [66]:
w.shape
w

TensorShape([Dimension(4), Dimension(1)])

<tf.Tensor: id=4505, shape=(4, 1), dtype=float32, numpy=
array([[-2.0675364e-09],
       [-3.9488079e-09],
       [-3.8743018e-09],
       [-2.9802314e-09]], dtype=float32)>

In [67]:
b.shape
b

TensorShape([Dimension(1)])

<tf.Tensor: id=4508, shape=(1,), dtype=float32, numpy=array([4.788385], dtype=float32)>

### Model Prediction on 1st Examples in Test Dataset

In [68]:
w
b

<tf.Tensor: id=4505, shape=(4, 1), dtype=float32, numpy=
array([[-2.0675364e-09],
       [-3.9488079e-09],
       [-3.8743018e-09],
       [-2.9802314e-09]], dtype=float32)>

<tf.Tensor: id=4508, shape=(1,), dtype=float32, numpy=array([4.788385], dtype=float32)>

In [69]:
current_prediction = prediction(test_x, w, b)

current_prediction[0]

<tf.Tensor: id=4519, shape=(1,), dtype=float32, numpy=array([4.788385], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load tensorflow if not done already

### Load the given Iris data using pandas (Iris.csv)

In [4]:
iris_df = pd.read_csv('Iris.csv')

In [5]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


In [6]:
iris_df.shape
iris_df.head()

(150, 6)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

### Splitting the data into feature set and target set

In [7]:
X = iris_df.iloc[:,1:5].values
y = iris_df.iloc[:,5].values

from sklearn.preprocessing import LabelEncoder
encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)

Y = pd.get_dummies(y1).values

### Divide the dataset into Training and test (70:30)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
print("X_train ", X_train.shape)
print("X_test ", X_test.shape)
print("Y_train ", Y_train.shape)
print("Y_test ", Y_test.shape)

X_train  (105, 4)
X_test  (45, 4)
Y_train  (105, 3)
Y_test  (45, 3)


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [9]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [10]:
# Create model
model = Sequential()

model.add(Dense(10,input_shape=(4,),activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(6,activation='relu'))
model.add(Dense(3,activation='softmax'))

#model.compile(Adam(lr=0.04),'categorical_crossentropy',metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 21        
Total params: 213
Trainable params: 213
Non-trainable params: 0
_________________________________________________________________


### Model Training 

In [11]:
#fitting the model and predicting 
model.fit(X_train,Y_train,epochs=100)


Instructions for updating:
Use tf.cast instead.
Epoch 1/100
105/105 [==============================] - 0s 2ms/step - loss: 1.1513 - acc: 0.3429
Epoch 2/100
105/105 [==============================] - 0s 76us/step - loss: 1.1340 - acc: 0.3429
Epoch 3/100
105/105 [==============================] - 0s 66us/step - loss: 1.1261 - acc: 0.3429
Epoch 4/100
105/105 [==============================] - 0s 95us/step - loss: 1.1188 - acc: 0.3429
Epoch 5/100
105/105 [==============================] - 0s 66us/step - loss: 1.1145 - acc: 0.3429
Epoch 6/100
105/105 [==============================] - 0s 85us/step - loss: 1.1108 - acc: 0.3333
Epoch 7/100
105/105 [==============================] - 0s 66us/step - loss: 1.1079 - acc: 0.3333
Epoch 8/100
105/105 [==============================] - 0s 104us/step - loss: 1.1054 - acc: 0.3238
Epoch 9/100
105/105 [==============================] - 0s 76us/step - loss: 1.1035 - acc: 0.3238
Epoch 10/100
105/105 [==============================] - 0s 76us/step - loss: 1.

105/105 [==============================] - 0s 67us/step - loss: 0.9876 - acc: 0.6952
Epoch 82/100
105/105 [==============================] - 0s 86us/step - loss: 0.9832 - acc: 0.6952
Epoch 83/100
105/105 [==============================] - 0s 95us/step - loss: 0.9791 - acc: 0.6952
Epoch 84/100
105/105 [==============================] - 0s 67us/step - loss: 0.9742 - acc: 0.6952
Epoch 85/100
105/105 [==============================] - 0s 76us/step - loss: 0.9687 - acc: 0.6952
Epoch 86/100
105/105 [==============================] - 0s 66us/step - loss: 0.9634 - acc: 0.6952
Epoch 87/100
105/105 [==============================] - 0s 85us/step - loss: 0.9581 - acc: 0.6952
Epoch 88/100
105/105 [==============================] - 0s 85us/step - loss: 0.9532 - acc: 0.6952
Epoch 89/100
105/105 [==============================] - 0s 76us/step - loss: 0.9472 - acc: 0.6952
Epoch 90/100
105/105 [==============================] - 0s 95us/step - loss: 0.9405 - acc: 0.6952
Epoch 91/100
105/105 [===========

In [12]:
#model.fit(X_train, Y_train, validation_split=0.3, epochs=10, batch_size=1)

### Model Prediction

In [13]:
from sklearn import metrics

In [14]:
y_pred = model.predict(X_test)

y_test_class = np.argmax(Y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)

In [15]:
#y_predict=model.predict(X_test)
print(metrics.accuracy_score(y_test_class,y_pred_class))


0.6


In [16]:
#Accuracy of the predicted values
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test_class,y_pred_class))
print(confusion_matrix(y_test_class,y_pred_class))

             precision    recall  f1-score   support

          0       0.93      1.00      0.97        14
          1       0.00      0.00      0.00        18
          2       0.43      1.00      0.60        13

avg / total       0.42      0.60      0.48        45

[[14  0  0]
 [ 1  0 17]
 [ 0  0 13]]


### Save the Model

In [18]:
model.save('internal_tf_keras.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?